In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [2]:
file_path_data = r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\train_data.csv"
file_path_labels = r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\train_labels.csv"

data = pd.read_csv(file_path_data, nrows = 1440000)

labels = pd.read_csv(file_path_labels)

NameError: name 'pd' is not defined

In [4]:
print(data.head)

<bound method NDFrame.head of                                                customer_ID         S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  2017-07-16   
...                                                    ...         ...   
1439995  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-06-29   
1439996  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-07-29   
1439997  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-08-30   
1439998  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-09-29   
1439999  425d5532a25f20815166956bc7fb3a49928f4c49517e1c...  2017-10-30   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \


In [5]:
data['S_2'] = pd.to_datetime(data['S_2'])
#data = data.groupby('customer_ID')
data.tail(5)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
1439995,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-06-29,0.874078,0.004086,0.008672,0.815791,0.005246,NaN,0.008112,0.006125,...,NaN,NaN,NaN,0.002196,0.006976,0.004546,NaN,0.009514,0.002603,0.009218
1439996,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-07-29,0.872043,0.007395,0.001348,0.819464,0.004784,NaN,0.009488,0.008950,...,NaN,NaN,NaN,0.002335,0.005112,0.003713,NaN,0.000811,0.008883,0.001153
1439997,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-08-30,0.867252,0.008150,0.008059,0.818707,0.007366,NaN,0.005170,0.000103,...,NaN,NaN,NaN,0.005987,0.008728,0.004927,NaN,0.007416,0.006252,0.002668
1439998,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-09-29,0.934603,0.002312,0.006102,0.816108,0.003532,NaN,0.000250,0.000844,...,NaN,NaN,NaN,0.006903,0.004097,0.009254,NaN,0.005879,0.002002,0.005971
1439999,425d5532a25f20815166956bc7fb3a49928f4c49517e1c...,2017-10-30,0.938461,0.005231,0.000666,0.810569,0.007312,NaN,0.004932,0.002325,...,NaN,NaN,NaN,0.008898,0.006947,0.008566,NaN,0.001507,0.001307,0.009375


We must have complete data for each customer, so we will remove the last customer ID from the data

In [6]:
last_customer_id = data['customer_ID'].iloc[-1]
print(last_customer_id)

425d5532a25f20815166956bc7fb3a49928f4c49517e1c59f9843f13a8aad690


Now we will remove this customer ID from the dataset, so that we only have complete information for all the customers.

In [7]:
data2 = data[data['customer_ID'] != '425d5532a25f20815166956bc7fb3a49928f4c49517e1c59f9843f13a8aad690']
print(data2.tail)

<bound method NDFrame.tail of                                                customer_ID        S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-07-16   
...                                                    ...        ...   
1439987  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-11-02   
1439988  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-12-22   
1439989  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-01-18   
1439990  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-02-18   
1439991  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-03-12   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \
0        0.9

In [8]:
data2.shape

(1439992, 190)

Now we will merge the data and labels

In [63]:
merged_data = pd.merge(data2, labels, on='customer_ID').dropna(subset=['target'])
merged_data.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\merged-data-final.csv", index = False)

In [64]:
data_types = merged_data.dtypes
data_types.dtype
data_types = pd.DataFrame({"label": merged_data.columns,"type": data_types})
print(data_types)

                   label            type
customer_ID  customer_ID          object
S_2                  S_2  datetime64[ns]
P_2                  P_2         float64
D_39                D_39         float64
B_1                  B_1         float64
...                  ...             ...
D_142              D_142         float64
D_143              D_143         float64
D_144              D_144         float64
D_145              D_145         float64
target            target           int64

[191 rows x 2 columns]


In [11]:
#data_types.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\data_types.csv", index = False)

In [65]:
categorical_variables = data_types.loc[data_types['type']=='object']
print(categorical_variables)

                   label    type
customer_ID  customer_ID  object
D_63                D_63  object
D_64                D_64  object


In [66]:
categorical_variables = categorical_variables.iloc[1:]
print(categorical_variables)

     label    type
D_63  D_63  object
D_64  D_64  object


In [67]:
cat_list = categorical_variables.iloc[:, 0].tolist()

In [68]:
print(cat_list)

['D_63', 'D_64']


In [69]:
one_hot = OneHotEncoder(handle_unknown = 'ignore')
one_hot.fit(merged_data[cat_list])
merged_data_encoded = pd.DataFrame(one_hot.transform(merged_data[cat_list]).toarray(),columns = one_hot.get_feature_names_out(cat_list))
merged_data = pd.concat([merged_data.reset_index(drop = True), merged_data_encoded.reset_index(drop=True)],axis = 1)
print(merged_data)

                                               customer_ID        S_2  \
0        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-03-09   
1        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-04-07   
2        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-05-28   
3        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-06-13   
4        0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f... 2017-07-16   
...                                                    ...        ...   
1439987  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-11-02   
1439988  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2017-12-22   
1439989  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-01-18   
1439990  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-02-18   
1439991  425d3742a1fd67755c431a132257d18e81234fc36985cd... 2018-03-12   

              P_2      D_39       B_1       B_2       R_1       S_3      D_41  \
0        0.938469  0.001733  0.008724  1.0

Find the range of dates in the sample to determine test-train split

In [17]:
print(merged_data['S_2'].min())
print(merged_data['S_2'].max())

2017-03-01 00:00:00
2018-03-31 00:00:00


Split data into test train split.
Train sample will contain all data from 2017-05 to 2018-01 inclusive.
Test sample 1 will contain data from 2017-03 to 2017-04 inclusive and test sample 2 will contain data from 2018-02 to 2018-03.

In [18]:
train_data = merged_data[(merged_data['S_2']>='2017-05-01') & (merged_data["S_2"] <= '2018-01-31')]
test1_data = merged_data[(merged_data['S_2']>='2017-03-01') & (merged_data['S_2'] <= '2017-04-30')]
test2_data = merged_data[(merged_data['S_2']>='2018-02-01') & (merged_data['S_2'] <= '2018-03-31')]

Check to see if date ranges in train and test sample are correct

In [19]:
print(train_data['S_2'].min())
print(train_data['S_2'].max())
print(test1_data['S_2'].min())
print(test1_data['S_2'].max())
print(test2_data['S_2'].min())
print(test2_data['S_2'].max())

2017-05-01 00:00:00
2018-01-31 00:00:00
2017-03-01 00:00:00
2017-04-30 00:00:00
2018-02-01 00:00:00
2018-03-31 00:00:00


Drop columns not needed for the model and create X_train and Y_train dataframes, each containing the independent and dependent variables respectively.

In [20]:
target_col = 'target'
drop_cols = ['customer_ID','S_2', 'D_63','D_64',target_col]
X_train = train_data.drop(columns = drop_cols)
Y_train = train_data[target_col]

In [21]:
Y_train.sum()

249910

Now we are ready to run XGBoost on our data, we will initially use the default parameters

In [22]:
xgb_model_default = xgb.XGBClassifier(random_state = 42)
xgb_model_default.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

Calculate the feature importance and save it to a csv file

In [23]:
importance_df = pd.DataFrame(xgb_model_default.feature_importances_, index = X_train.columns, columns=['importance'])
importance_df = importance_df.sort_values('importance', ascending=False)

In [24]:
importance_df
importance_df.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\feature_importance.csv")

Run another XGBoost model using the parameters listed:
-Trees = 300
-Learning Rate = 0.5
-Max Depth = 4
-Observations per sample = 0.5
-weight = 5

In [25]:
params = {'learning_rate': 0.5,
          'max_depth': 4,
          'subsample': 0.5,
          'colsample_bytree':0.5,
          'scale_pos_weight': 5,
          'random_state': 42}

# Set the number of trees
num_trees = 300

# Build the XGBoost model
xgb_model_new = xgb.XGBClassifier(n_estimators=num_trees, **params)
xgb_model_new.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

Build a second feature importance dataframe and save it to csv

In [26]:
importance_df2 = pd.DataFrame(xgb_model_new.feature_importances_, index = X_train.columns, columns=['importance'])
importance_df2 = importance_df2.sort_values('importance', ascending=False)
print(importance_df2)

          importance
D_48        0.181197
B_9         0.081526
P_2         0.072415
D_75        0.055379
R_1         0.026201
...              ...
B_30        0.000194
B_31        0.000000
D_137       0.000000
D_63_XL     0.000000
D_64_nan    0.000000

[197 rows x 1 columns]


Save second feature importance dataframe to a second csv

In [27]:
importance_df2.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\feature_importance2.csv")

Merge 2 feature importance dataframes

In [28]:
# Merge the two dataframes on the index column
merged_importance_df = importance_df.merge(importance_df2, left_index=True, right_index=True)

# Rename the importance columns
merged_importance_df = merged_importance_df.rename(columns={'importance_x': 'importance_1', 'importance_y': 'importance_2'})

# Add a column for the feature names
merged_importance_df['feature_name'] = merged_importance_df.index

# Reorder the columns to have the feature_name column on the left
merged_importance_df = merged_importance_df[['feature_name', 'importance_1', 'importance_2']]

# Reset the index
merged_importance_df = merged_importance_df.reset_index(drop=True)

print(merged_importance_df)

    feature_name  importance_1  importance_2
0            P_2      0.203091      0.072415
1            B_1      0.071032      0.005521
2            B_2      0.063135      0.004818
3            B_9      0.046202      0.081526
4           D_51      0.030012      0.008512
..           ...           ...           ...
192         B_31      0.000000      0.000000
193        D_116      0.000000      0.000976
194        D_137      0.000000      0.000000
195         R_20      0.000000      0.000866
196     D_64_nan      0.000000      0.000000

[197 rows x 3 columns]


In [29]:
merged_importance_df.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\merged_importance.csv")

Create a list of features with importance greater than 0.5% in either model

In [30]:
# Create a boolean mask for features with importance greater than 0.5% in either column
mask = (merged_importance_df['importance_1'] > 0.005) | (merged_importance_df['importance_2'] > 0.005)

# Get the feature names where the mask is True
important_features = merged_importance_df.loc[mask, 'feature_name'].tolist()
print(len(important_features))

41


Remove all features with importance less than 0.5% (0.005)

In [31]:
# Filter the X_train dataframe based on the columns to keep
X_train_filtered = X_train[important_features]
print(X_train_filtered.shape)

(995264, 41)


In [32]:
X_test1 = test1_data.drop(columns = drop_cols)
X_test1 = X_test1[important_features]
Y_test1 = test1_data[target_col]
X_test2 = test2_data.drop(columns = drop_cols)
X_test2 = X_test2[important_features]
Y_test2 = test2_data[target_col]

#test to see if shapes of train and test samples are the same
print(X_train_filtered.shape)
print(Y_train.shape)
print(X_test1.shape)
print(X_test2.shape)
print(Y_test1.shape)
print(Y_test2.shape)

(995264, 41)
(995264,)
(207531, 41)
(237197, 41)
(207531,)
(237197,)


In [35]:
from sklearn.model_selection import ParameterGrid
clf = xgb.XGBClassifier(objective='binary:logistic', seed=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 300],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.5, 0.8],
    'colsample_bytree': [0.5, 1.0],
    'scale_pos_weight': [1, 5, 10]
}

# Initialize the grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Initialize the dataframe to store the results
results_table = pd.DataFrame(columns=["Trees", "LR", "Subsample", "% Features", "Weight of Default", "AUC Train", "AUC Test 1", "AUC Test 2"])

# Loop over all parameter combinations
for params in ParameterGrid(param_grid):
    # Set the parameters for the classifier
    clf.set_params(**params)
    
    # Fit the classifier to the training data
    clf.fit(X_train_filtered, Y_train)
    
    # Compute the AUC score on the training set
    y_train_pred = clf.predict_proba(X_train_filtered)[:, 1]
    auc_train = roc_auc_score(Y_train, y_train_pred)
    
    # Compute the AUC score on the first test set
    y_test1_pred = clf.predict_proba(X_test1)[:, 1]
    auc_test1 = roc_auc_score(Y_test1, y_test1_pred)
    
    # Compute the AUC score on the second test set
    y_test2_pred = clf.predict_proba(X_test2)[:, 1]
    auc_test2 = roc_auc_score(Y_test2, y_test2_pred)
    
    # Append the results to the dataframe
    results_table = results_table.append({
        "Trees": params['n_estimators'],
        "LR": params['learning_rate'],
        "Subsample": params['subsample'],
        "% Features": params['colsample_bytree'],
        "Weight of Default": params['scale_pos_weight'],
        "AUC Train": auc_train,
        "AUC Test 1": auc_test1,
        "AUC Test 2": auc_test2
    }, ignore_index=True)
    
    # Print the current results
    print(results_table.iloc[-1])

C:\Users\aniqa\AppData\Local\Temp\ipykernel_18896\4158414759.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


Trees                50.000000
LR                    0.010000
Subsample             0.500000
% Features            0.500000
Weight of Default     1.000000
AUC Train             0.927606
AUC Test 1            0.910497
AUC Test 2            0.944293
Name: 0, dtype: float64


C:\Users\aniqa\AppData\Local\Temp\ipykernel_18896\4158414759.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


Trees                50.000000
LR                    0.010000
Subsample             0.800000
% Features            0.500000
Weight of Default     1.000000
AUC Train             0.927579
AUC Test 1            0.910308
AUC Test 2            0.944188
Name: 1, dtype: float64


C:\Users\aniqa\AppData\Local\Temp\ipykernel_18896\4158414759.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


...

C:\Users\aniqa\AppData\Local\Temp\ipykernel_18896\4158414759.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


Trees                300.000000
LR                     0.100000
Subsample              0.800000
% Features             1.000000
Weight of Default      5.000000
AUC Train              0.948218
AUC Test 1             0.926494
AUC Test 2             0.956973
Name: 69, dtype: float64


C:\Users\aniqa\AppData\Local\Temp\ipykernel_18896\4158414759.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


Trees                300.000000
LR                     0.100000
Subsample              0.500000
% Features             1.000000
Weight of Default     10.000000
AUC Train              0.946772
AUC Test 1             0.925421
AUC Test 2             0.955943
Name: 70, dtype: float64
Trees                300.000000
LR                     0.100000
Subsample              0.800000
% Features             1.000000
Weight of Default     10.000000
AUC Train              0.947885
AUC Test 1             0.926134
AUC Test 2             0.956620
Name: 71, dtype: float64


C:\Users\aniqa\AppData\Local\Temp\ipykernel_18896\4158414759.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({


In [36]:
results_table.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\XGB_GridSearch_results.csv")

In [4]:
results_table = pd.read_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\XGB_GridSearch_results.csv")
results_table

,Trees,LR,Subsample,% Features,Weight of Default,AUC Train,AUC Test 1,AUC Test 2
0,50,0.01,0.5,0.5,1,0.927606,0.910497,0.944293
1,50,0.01,0.8,0.5,1,0.927579,0.910308,0.944188
2,50,0.01,0.5,0.5,5,0.926159,0.909704,0.942208
3,50,0.01,0.8,0.5,5,0.926024,0.909482,0.942005
4,50,0.01,0.5,0.5,10,0.924595,0.908080,0.940674
...,...,...,...,...,...,...,...,...
67,300,0.10,0.8,1.0,1,0.948558,0.926493,0.957471
68,300,0.10,0.5,1.0,5,0.947488,0.925983,0.956505
69,300,0.10,0.8,1.0,5,0.948218,0.926494,0.956973
70,300,0.10,0.5,1.0,10,0.946772,0.925421,0.955943


In [5]:
#Calculate the average AUC across test samples
results_table = results_table.assign(Average_AUC = np.mean(results_table[['AUC Train','AUC Test 1', 'AUC Test 2']], axis=1))

# Calculate the variance of AUC across test samples
results_table = results_table.assign(AUC_Variance = np.var(results_table[['AUC Train','AUC Test 1', 'AUC Test 2']], axis=1))
results_table

,Trees,LR,Subsample,% Features,Weight of Default,AUC Train,AUC Test 1,AUC Test 2,Average_AUC,AUC_Variance
0,50,0.01,0.5,0.5,1,0.927606,0.910497,0.944293,0.927466,0.000190
1,50,0.01,0.8,0.5,1,0.927579,0.910308,0.944188,0.927358,0.000191
2,50,0.01,0.5,0.5,5,0.926159,0.909704,0.942208,0.926023,0.000176
3,50,0.01,0.8,0.5,5,0.926024,0.909482,0.942005,0.925837,0.000176
4,50,0.01,0.5,0.5,10,0.924595,0.908080,0.940674,0.924450,0.000177
...,...,...,...,...,...,...,...,...,...,...
67,300,0.10,0.8,1.0,1,0.948558,0.926493,0.957471,0.944174,0.000170
68,300,0.10,0.5,1.0,5,0.947488,0.925983,0.956505,0.943325,0.000164
69,300,0.10,0.8,1.0,5,0.948218,0.926494,0.956973,0.943895,0.000164
70,300,0.10,0.5,1.0,10,0.946772,0.925421,0.955943,0.942712,0.000164


In [6]:
results_table.to_csv(r"C:\Users\aniqa\Desktop\Courses\Semester_2\Machine_Learning\Project 1\Code\XGB_GridSearch_results_final.csv")

In [54]:
bestparams = {'learning_rate': 0.1,
          'subsample': 0.5,
          'colsample_bytree':0.5,
          'scale_pos_weight': 10,
          'random_state': 42}

# Set the number of trees
num_trees = 300

# Build the XGBoost model
xgb_model_best = xgb.XGBClassifier(n_estimators=num_trees, **bestparams)
xgb_model_best.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)